In [41]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [44]:
def define_paths(data_dir):
    filepaths = []
    labels = []
    folds = listdir_nohidden(data_dir)
    for fold in folds:
        fold_path = os.path.join(data_dir, fold)
        files = listdir_nohidden(fold_path)
        for file in files:
            fpath = os.path.join(fold_path, file)
            filepaths.append(fpath)
            labels.append(fold)
    return filepaths, labels

def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

def create_df(data_dir):
    files, classes = define_paths(data_dir)
    df = define_df(files, classes)
    
    strat = df['labels']
    train_df, test_df = train_test_split(df,  train_size= 0.9, shuffle= True, random_state= 123, stratify= strat)
 
    return train_df, test_df

In [45]:
def define_paths(data_dir):
    filepaths = []
    labels = []
    folds = listdir_nohidden(data_dir)
    for fold in folds:
        fold_path = os.path.join(data_dir, fold)
        files = listdir_nohidden(fold_path)
        for file in files:
            fpath = os.path.join(fold_path, file)
            filepaths.append(fpath)
            labels.append(fold)
    return filepaths, labels

def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)

def create_df(data_dir):
    files, classes = define_paths(data_dir)
    df = define_df(files, classes)
    
    strat = df['labels']
    train_df, test_df = train_test_split(df,  train_size= 0.9, shuffle= True, random_state= 123, stratify= strat)
 
    return train_df, test_df

In [46]:
data_dir = 'Data/bone_marrow_cell_dataset'
train_df, test_df = create_df(data_dir)

In [47]:
train_df.head()

,filepaths,labels
137194,Data/bone_marrow_cell_dataset/NGS/NGS_08972.jpg,NGS
68587,Data/bone_marrow_cell_dataset/EBO/EBO_18417.jpg,EBO
92767,Data/bone_marrow_cell_dataset/BLA/BLA_10116.jpg,BLA
20834,Data/bone_marrow_cell_dataset/ART/ART_13016.jpg,ART
68868,Data/bone_marrow_cell_dataset/EBO/EBO_19251.jpg,EBO


In [48]:
print(len(train_df['labels'].unique()))

14


In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_gens(train_df, test_df):
    img_size = (250, 250)
    channels = 3
    img_shape = (img_size[0], img_size[1], channels)
    batch_size = 40
    ts_length = len(test_df)
    test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
    test_steps = ts_length // test_batch_size
    def scalar(img):
        return img
    tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
    ts_gen = ImageDataGenerator(preprocessing_function= scalar)
    train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= True, batch_size= batch_size)
    test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= False, batch_size= test_batch_size)
    return train_gen, test_gen

In [50]:
train_gen, test_gen = create_gens(train_df, test_df)

Found 153052 validated image filenames belonging to 14 classes.
Found 17006 validated image filenames belonging to 14 classes.


In [51]:
train_gen

In [52]:
num_classes = 14

In [53]:
classifier_cnn = tf.keras.Sequential([
    # first conv
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    # second conv
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.1),
    # second conv
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.1),
    # ----
    tf.keras.layers.Flatten(),
    # dense layers
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    # classifier
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

In [54]:
classifier_cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [55]:
classifier_cnn.fit_generator(train_gen,
                         steps_per_epoch = 50,
                         epochs = 10,
                         validation_data = test_gen,
                         validation_steps = 500)

Epoch 1/10


/var/folders/5g/b5hghd9x6gq7f3qsrm03q_r80000gn/T/ipykernel_90386/35442443.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier_cnn.fit_generator(train_gen,


50/50 [==============================] - ETA: 0s - loss: 0.2739 - accuracy: 0.2370

KeyboardInterrupt: 

Validation accuracy reaches about 61% with 25 epochs.

There is a few corrupted files/ known issue with openCV that made it impossible to run evaluate. Going to change train/test datasets.

 599/5669 [==>...........................] - ETA: 44s - loss: 0.1569 - accuracy: 0.5415

2022-12-10 12:32:24.145525: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: TypeError: float() argument must be a string or a number, not 'JpegImageFile'
Traceback (most recent call last):

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/keras/engine/data_adapter.py", line

InvalidArgumentError: Graph execution error:

TypeError: float() argument must be a string or a number, not 'JpegImageFile'
Traceback (most recent call last):

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "/Users/emmanuelpeters/miniforge3/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 956, in generator_fn
    yield x[i]

  File "/

In [56]:
import cv2 as cv
import glob


file_paths = test_df['filepaths']
for file_path in file_paths:
    try:
        img = cv.imread(file_path)
    except:
        print(file_path)

Couldn't fix it with code, used https://github.com/coderslagoon/BadPeggy to clean up data.

In [ ]:

from PIL import Image

num_skipped = 0

folder_path = test_df['filepaths']
for fpath in folder_path:
    try:
        img = Image.open(fpath) # open the image file
        #print("%s", fpath)
        exif_data = img._getexif()
        img.verify() # verify that it is, in fact an image
    except:
        num_skipped += 1
        # Delete corrupted image
        os.remove(fpath)
print("PIL deleted %d images" % num_skipped)

PIL deleted 0 images


In [57]:
img_size = (250, 250)
def scalar(img):
        return img
ts_gen = ImageDataGenerator(preprocessing_function= scalar)
test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                        color_mode= 'rgb', shuffle= False)

Found 17006 validated image filenames belonging to 14 classes.


In [58]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory= 'Data/bone_marrow_cell_dataset/',
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=img_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = 'Data/bone_marrow_cell_dataset/',
    validation_split=0.2,
    subset="validation",
    label_mode='categorical',
    seed=1337,
    image_size=img_size,
)

Found 170058 files belonging to 14 classes.
Using 136047 files for training.
Found 170058 files belonging to 14 classes.
Using 34011 files for validation.


In [59]:
classifier_cnn.fit(train_ds,
                         steps_per_epoch = 50,
                         epochs = 10,
                         validation_data = val_ds,
                         validation_steps = 500)

Epoch 1/10
50/50 [==============================] - 23s 462ms/step - loss: 0.2143 - accuracy: 0.3269 - val_loss: 0.2278 - val_accuracy: 0.3124
Epoch 2/10
50/50 [==============================] - 23s 458ms/step - loss: 0.1959 - accuracy: 0.4075 - val_loss: 0.2043 - val_accuracy: 0.3738
Epoch 3/10
50/50 [==============================] - 24s 482ms/step - loss: 0.1880 - accuracy: 0.4363 - val_loss: 0.2568 - val_accuracy: 0.2718
Epoch 4/10
50/50 [==============================] - 26s 533ms/step - loss: 0.1809 - accuracy: 0.4550 - val_loss: 0.1883 - val_accuracy: 0.4366
Epoch 5/10
50/50 [==============================] - 25s 503ms/step - loss: 0.1737 - accuracy: 0.4875 - val_loss: 0.2017 - val_accuracy: 0.4267
Epoch 6/10
50/50 [==============================] - 25s 508ms/step - loss: 0.1714 - accuracy: 0.4888 - val_loss: 0.2479 - val_accuracy: 0.3140
Epoch 7/10
50/50 [==============================] - 25s 516ms/step - loss: 0.1638 - accuracy: 0.5175 - val_loss: 0.2121 - val_accuracy: 0.4354

In [ ]:
num_skipped = 0

for fname in test_df['filepaths']:
    fpath = fname
    try:
        fobj = open(fpath, "rb")
        is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
    finally:
        fobj.close()

    if not is_jfif:
        num_skipped += 1
        print("delete")
        # Delete corrupted image
        os.remove(fpath)

In [75]:
y_pred = classifier_cnn.predict(val_ds)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [67]:
val_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 250, 250, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 14), dtype=tf.float32, name=None))>

In [80]:
y_true = np.concatenate([y for x, y in val_ds], axis=0)
y_true = y_true.argmax(axis=-1)


In [81]:
y_pred = y_pred.argmax(axis=-1)


In [85]:
print(np.unique(y_pred))

[ 0  1  2  3  4  7  8  9 11 12 13]


In [86]:
print(np.unique(y_true))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


In [91]:
classifier_cnn.evaluate(val_ds)

1063/1063 [==============================] - 39s 37ms/step - loss: 0.2178 - accuracy: 0.3926


[0.21776337921619415, 0.39260828495025635]

In [92]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(250, 250, 3),
    include_top=False)

83697664/83683744 [==============================] - 3s 0us/step


In [102]:
classifier_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, None, None, 64)    1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, None, None, 64)   0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, None, None, 64)   256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, None, None, 64)    0         
                                                                 
 conv2d_4 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, None, None, 64)  

 # first conv
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    # second conv
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.1),
    # second conv
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=(2, 2), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.1),
    # ----
    tf.keras.layers.Flatten(),
    # dense layers
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    # classifier
    tf.keras.layers.Dense(num_classes, activation='softmax'),

In [109]:
base_model.trainable = False

In [110]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250, 250, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 124, 124, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 124, 124, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

In [111]:
inputs = tf.keras.Input(shape=(250, 250, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=3,  activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(.3)(x)
# second conv
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(.1)(x)
# second conv
x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(.1)(x)
# ----
x = tf.keras.layers.Flatten()(x)
# dense layers
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
# classifier
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
new_model = tf.keras.Model(inputs, outputs)

In [112]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 conv2d_15 (Conv2D)          (None, 8, 8, 64)          1179712   
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 4, 4, 64)         0         
 g2D)                                                            
                                                                 
 batch_normalization_17 (Bat  (None, 4, 4, 64)         256       
 chNormalization)                                                
                                                                 
 dropout_13 (Dropout)        (None, 4, 4, 64)          0   

In [113]:
new_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [114]:
new_model.fit(train_ds, validation_data=val_ds)

  17/4252 [..............................] - ETA: 1:28:07 - loss: 0.3815 - accuracy: 0.1636